In [1]:
import pymysql
import pandas as pd

db_config = {
    "host": "database-1.cvg820c68twg.ap-south-1.rds.amazonaws.com",
    "user": "admin",
    "password": "Password!23",
    "database": "galen_chat"
}

conn = pymysql.connect(**db_config)
query = "SELECT epad_text,speciality FROM clinisquare_db WHERE epad_text IS NOT NULL"

df = pd.read_sql(query, conn)

df.to_csv("prescription_data/prescriptions_raw.csv", index=False)

conn.close()
print("Prescription data extracted successfully!")


/var/folders/sw/mplph8ks42g1krcbdjk6xg8c0000gn/T/ipykernel_76525/448611605.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Prescription data extracted successfully!


In [2]:
import pandas as pd

df = pd.read_csv("prescription_data/prescriptions_raw.csv")
print(df.head())


                                           epad_text         speciality
0  ID: #P22S404 | Neeta Lal (female , 37) Date: M...        Orthopedics
1  OPD Note ID: #P28S01 | Name: Rohit Jha | Age: ...    Family Medicine
2  Untitled ID: #P07S01 | Name: vikas natekar | A...         Pediatrics
3  OPD Note ID: #P04S01 | Name: Kabir Sharma | Ag...  Internal Medicine
4  Clinical Notes ID: #P24S127 | Aditya Singh (ma...        Dermatology


In [3]:
df = df[~df['epad_text'].astype(str).str.startswith("Patient History Form ID:")]
df = df[~df['epad_text'].astype(str).str.startswith("Informed Consent Form ID:")]
df = df[~df['epad_text'].astype(str).str.startswith("Medical Certificate")]
df = df[~df['epad_text'].astype(str).str.startswith("Certificate")]
# df = df[~df['epad_text'].astype(str).str.startswith("OPD Note  | Location: ")]

df

,epad_text,speciality
0,"ID: #P22S404 | Neeta Lal (female , 37) Date: M...",Orthopedics
1,OPD Note ID: #P28S01 | Name: Rohit Jha | Age: ...,Family Medicine
2,Untitled ID: #P07S01 | Name: vikas natekar | A...,Pediatrics
3,OPD Note ID: #P04S01 | Name: Kabir Sharma | Ag...,Internal Medicine
4,Clinical Notes ID: #P24S127 | Aditya Singh (ma...,Dermatology
...,...,...
3260,"ID: #P22S406 | Mina Sharma (female , 70) Date:...",Orthopedics
3263,"ID: #P22S447 | Shalu Khandelwal (female , 39) ...",Orthopedics
3264,"ID: #P03S07 | Prabhmeet P (female , 43) Date: ...",Physiotherapy
3270,ID: #P198S24 | Chirag Shah (Male ) Date: Apr 0...,Sports Medicine and Arthroplasty


In [4]:
import re

def clean_text(text):
    if pd.isna(text):
        return text  
    
    text = re.sub(r"ID: #\S+ \| .*? Date: .*?\d{2}:\d{2} (AM|PM)", "", text)
    text = re.sub(r"OPD Note ID: #\S+ \| Name: .*? \| Age: \d+ \| Gender: (male|female) \| .*?", "", text)
    text = re.sub(r"^(Clinical Notes|Prescription|Follow-up:|Vitals)\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"^(?:\d{1,2}(st|nd|rd|th)?\s+[A-Za-z]+\s+\d{2,4})", "", text)
    text = re.sub(r"[●*]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.strip()

df['epad_text'] = df['epad_text'].astype(str).apply(clean_text)
df = df.dropna(subset=['epad_text'])

In [5]:
df.to_csv("prescription_data/prescription_clean.csv", index=False)

In [ ]:
import re

date_pattern = r"\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2},? \d{2,4})\b"
id_pattern = r"ID: #[A-Za-z0-9]+"
meta_patterns = [r"Visit Date: .*?\|", r"Location: .*?\|", r"Name: .*?\|", r"Age: \d+ \|", r"Gender: .*?\|"]

def clean_text(text):
    text = text.lower()  #lowercase
    text = re.sub(date_pattern, "", text)  #dates
    text = re.sub(id_pattern, "", text)  #IDs
    for pattern in meta_patterns:
        text = re.sub(pattern, "", text)  #metadata
    text = re.sub(r"\d+", "", text)  #standalone numbers
    text = re.sub(r"[^\w\s]", "", text)  #special characters
    text = re.sub(r"\s+", " ", text).strip()  #extra spaces
    return text

df["epad_text_clean"] = df["epad_text"].astype(str).apply(clean_text)

df.to_csv("data/prescription_data/prescriptions_clean.csv", index=False)

print("Data cleaning complete!")


Data cleaning complete!


In [ ]:
import nltk
from nltk.util import ngrams
from collections import Counter

nltk.download("punkt")

df["tokenized"] = df["epad_text_clean"].apply(nltk.word_tokenize)

unigrams = []
bigrams = []
trigrams = []

for tokens in df["tokenized"]:
    unigrams.extend(tokens)
    bigrams.extend(ngrams(tokens, 2))
    trigrams.extend(ngrams(tokens, 3))

common_unigrams = Counter(unigrams).most_common(500)
common_bigrams = Counter(bigrams).most_common(500)
common_trigrams = Counter(trigrams).most_common(500)

df_unigrams = pd.DataFrame(common_unigrams, columns=["word", "count"])
df_bigrams = pd.DataFrame(common_bigrams, columns=["phrase", "count"])
df_trigrams = pd.DataFrame(common_trigrams, columns=["phrase", "count"])

df_unigrams.to_csv("data/prescription_data/common_unigrams.csv", index=False)
df_bigrams.to_csv("data/prescription_data/common_bigrams.csv", index=False)
df_trigrams.to_csv("data/prescription_data/common_trigrams.csv", index=False)

print("Tokenization & phrase extraction complete!")


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


Tokenization & phrase extraction complete!


In [ ]:
import json

unigrams = pd.read_csv("data/prescription_data/common_unigrams.csv")
bigrams = pd.read_csv("data/prescription_data/common_bigrams.csv")
trigrams = pd.read_csv("data/prescription_data/common_trigrams.csv")

phrase_dict = {
    "unigrams": unigrams.set_index("word").to_dict()["count"],
    "bigrams": bigrams.set_index("phrase").to_dict()["count"],
    "trigrams": trigrams.set_index("phrase").to_dict()["count"]
}

with open("data/prescription_data/phrase_lookup.json", "w") as f:
    json.dump(phrase_dict, f)

print("Phrase lookup dictionary saved!")


Phrase lookup dictionary saved!


In [ ]:
import pandas as pd

df_clean = pd.read_csv("data/prescription_data/prescriptions_clean.csv")

df_clean["epad_text_clean"] = df_clean["epad_text_clean"].astype(str).fillna("")

train_data = []
for text in df_clean["epad_text_clean"]:
    words = text.split()
    if len(words) < 2:
        continue  
    for i in range(len(words) - 1):
        train_data.append((" ".join(words[:i+1]), words[i+1]))  # (context, next_word)

df_train = pd.DataFrame(train_data, columns=["context", "next_word"])

df_train.to_csv("data/prescription_data/training_data.csv", index=False)

print("✅ Training data created successfully!")


✅ Training data created successfully!


N-GRAM Model training

In [ ]:
import nltk
from nltk import FreqDist
from nltk.util import ngrams
import pandas as pd
import random

df_train = pd.read_csv("data/prescription_data/training_data.csv")

unigrams = []
bigrams = []
trigrams = []

for index, row in df_train.iterrows():
    words = row["context"].split()
    unigrams.extend(words)
    bigrams.extend(ngrams(words, 2))
    trigrams.extend(ngrams(words, 3))

unigram_freq = FreqDist(unigrams)
bigram_freq = FreqDist(bigrams)
trigram_freq = FreqDist(trigrams)

unigram_prob = {word: unigram_freq[word] / sum(unigram_freq.values()) for word in unigram_freq}
bigram_prob = {word: bigram_freq[word] / sum(bigram_freq.values()) for word in bigram_freq}
trigram_prob = {word: trigram_freq[word] / sum(trigram_freq.values()) for word in trigram_freq}

import pickle
with open("models/ngram_model.pkl", "wb") as f:
    pickle.dump({"unigram": unigram_prob, "bigram": bigram_prob, "trigram": trigram_prob}, f)

print("N-Gram model trained and saved!")
